In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import numpy as np
import pandas as pd 
from sklearn import preprocessing 
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier

In [3]:
import pandas as pd
path = "/content/drive/MyDrive/newpro/threeclass_problem/ANOVA_Bert/data.csv"
df = pd.read_csv(path)
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,Level 5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,Level 5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,Level 5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",Level 5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Level 2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [4]:
df.iloc[:,1].replace(['Level 1', 'Level 2', 'Level 3', 'Level 4', 'Level 5'],
                        [1, 2, 3, 4, 5], inplace=True)

In [5]:
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [6]:
#define bins
bins = [0,2,4,5]
#name of the group 
group_names = ['Low', 'Moderate', 'High']
df['Level '] = pd.cut(df["Level "],bins,labels = group_names)

In [7]:
df.head(5)

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,High,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,High,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,High,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",High,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Low,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [8]:
df['Level '] = df['Level '].map({'Low': 1, 'Moderate': 2, 'High': 3}).astype(int)

In [9]:
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,3,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,3,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,3,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",3,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,1,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [10]:
import pandas as pd
path = "/content/drive/MyDrive/newpro/threeclass_problem/ANOVA_Bert/bert_withstopwordchangelevels_100_features.csv"
x = pd.read_csv(path)
x.head() 

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,90,91,92,93,94,95,96,97,98,99
0,0,-0.800343,1.511551,0.429687,0.496351,0.511075,1.236754,1.211082,0.220388,-1.556129,...,0.418513,0.594158,-0.542770,-1.304332,-0.167952,1.264030,1.248736,-0.795129,0.497081,-1.320194
1,1,0.202415,1.043170,-0.706025,-0.809822,0.544102,-1.218872,-0.489940,0.848106,-1.629868,...,-0.641821,-0.381261,-0.602636,1.109730,0.347108,-0.089155,0.036735,0.807292,-0.392206,0.366935
2,2,-0.649561,-0.313268,0.870955,0.869980,0.473374,0.511151,1.178802,-0.881936,1.330585,...,0.318945,0.858354,-1.047596,0.211807,-1.625572,0.992441,1.087589,-0.532345,1.797256,-0.804480
3,3,-0.948914,0.518390,0.907192,0.660595,0.991766,0.559171,0.493336,-1.306077,0.552494,...,0.780101,0.955115,-0.821807,-1.016353,-1.312187,0.460628,0.850170,-0.881615,1.194610,-0.908606
4,4,2.479892,-1.724730,-1.115911,-0.933822,-3.007723,-0.073396,0.848926,1.797432,1.014453,...,-1.436395,0.849854,2.075581,0.882456,2.213594,-2.516606,-1.140899,1.590177,-0.471105,1.280213


In [11]:
x.drop(x.columns[[0]], axis=1, inplace=True)
x.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.800343,1.511551,0.429687,0.496351,0.511075,1.236754,1.211082,0.220388,-1.556129,-1.298156,...,0.418513,0.594158,-0.542770,-1.304332,-0.167952,1.264030,1.248736,-0.795129,0.497081,-1.320194
1,0.202415,1.043170,-0.706025,-0.809822,0.544102,-1.218872,-0.489940,0.848106,-1.629868,-0.311534,...,-0.641821,-0.381261,-0.602636,1.109730,0.347108,-0.089155,0.036735,0.807292,-0.392206,0.366935
2,-0.649561,-0.313268,0.870955,0.869980,0.473374,0.511151,1.178802,-0.881936,1.330585,-0.645855,...,0.318945,0.858354,-1.047596,0.211807,-1.625572,0.992441,1.087589,-0.532345,1.797256,-0.804480
3,-0.948914,0.518390,0.907192,0.660595,0.991766,0.559171,0.493336,-1.306077,0.552494,-0.844049,...,0.780101,0.955115,-0.821807,-1.016353,-1.312187,0.460628,0.850170,-0.881615,1.194610,-0.908606
4,2.479892,-1.724730,-1.115911,-0.933822,-3.007723,-0.073396,0.848926,1.797432,1.014453,0.312498,...,-1.436395,0.849854,2.075581,0.882456,2.213594,-2.516606,-1.140899,1.590177,-0.471105,1.280213


In [12]:
y=df['Level '] 

**Train Test Split**

In [13]:
# Train-Test-Split is performed

from sklearn.model_selection import train_test_split

# Splitting the data into train and test
X_train, X_test, y_train, y_test = train_test_split(x, y, train_size=0.7, random_state=42)
X_train.shape, X_test.shape

((1220, 100), (524, 100))

In [14]:
print(y_train.value_counts(normalize=True)*100)
print("\n")
print(y_test.value_counts(normalize=True)*100)

2    45.327869
1    29.590164
3    25.081967
Name: Level , dtype: float64


2    45.229008
1    29.961832
3    24.809160
Name: Level , dtype: float64


**Model Building**

In [ ]:
def perceptron_classifier (X_train, y_train, X_test, y_test):
  p = Perceptron(random_state=150)
  p.fit(X_train, y_train)
  predictions_train = p.predict(X_train)
  predictions_test = p.predict(X_test)
  train_score = accuracy_score(predictions_train, y_train)
  #print("score on train data: ", train_score)
  test_score = f1_score(predictions_test, y_test, average='weighted' )
  print("Perceptron F1 score:", test_score)

In [ ]:
from sklearn.linear_model import Perceptron
perceptron_classifier(X_train, y_train, X_test, y_test) 

Perceptron F1 score: 0.36973517109054715


**Decision Tree**

In [15]:
clf_dt = DecisionTreeClassifier(max_depth=5, class_weight = "balanced")

In [16]:
clf_dt.fit(X_train,y_train)

DecisionTreeClassifier(class_weight='balanced', max_depth=5)

In [17]:
y_pred_train = clf_dt.predict(X_train)
y_pred_test = clf_dt.predict(X_test)

In [18]:
def classifcation_report_train_test(y_train, y_train_pred, y_test, y_test_pred):

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TRAIN DATA
            =========================================
            ''')
    print(classification_report(y_train, y_train_pred, digits=4))

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TEST DATA
            =========================================
            ''')
    print(classification_report(y_test, y_test_pred, digits=4))

In [19]:
scores = pd.DataFrame(columns=['Model', 'Train_Accuracy', 'Train_Recall', 'Train_Precision', 'Train_F1_Score', 
                               'Test_Accuracy', 'Test_Recall', 'Test_Precision', 'Test_F1_Score'])

def get_metrics(train_actual, train_predicted, test_actual, test_predicted, model_description, dataframe):

    train_accuracy  = accuracy_score(train_actual, train_predicted)
    train_recall    = recall_score(train_actual, train_predicted, average="weighted")
    train_precision = precision_score(train_actual, train_predicted, average="weighted")
    train_f1score   = f1_score(train_actual, train_predicted, average="weighted")
    
    test_accuracy   = accuracy_score(test_actual, test_predicted)
    test_recall     = recall_score(test_actual, test_predicted, average="weighted")
    test_precision  = precision_score(test_actual, test_predicted, average="weighted")
    test_f1score    = f1_score(test_actual, test_predicted, average="weighted")

    dataframe       = dataframe.append(pd.Series([model_description, 
                                                  train_accuracy, train_recall, train_precision, train_f1score,
                                                  test_accuracy, test_recall, test_precision, test_f1score],
                                                 index=scores.columns ), 
                                       ignore_index=True)

    return(dataframe)

In [20]:
scores = get_metrics(y_train, y_pred_train, y_test, y_pred_test, "DecisionTrees", scores)
scores

<ipython-input-19-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.388525,0.388525,0.619293,0.318029,0.312977,0.312977,0.34368,0.232598


**Random Forest**

In [21]:
clf_rf = RandomForestClassifier(class_weight = "balanced")

In [22]:
clf_rf.fit(X=X_train, y=y_train)

RandomForestClassifier(class_weight='balanced')

In [23]:
y_pred_train1 = clf_rf.predict(X_train)
y_pred_test1 = clf_rf.predict(X_test)

In [24]:
classifcation_report_train_test(y_train, y_pred_train1, y_test, y_pred_test1)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     1.0000    1.0000    1.0000       361
           2     1.0000    1.0000    1.0000       553
           3     1.0000    1.0000    1.0000       306

    accuracy                         1.0000      1220
   macro avg     1.0000    1.0000    1.0000      1220
weighted avg     1.0000    1.0000    1.0000      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2619    0.1401    0.1826       157
           2     0.4519    0.7722    0.5701       237
           3     0.2286    0.0615    0.0970       130

    accuracy                         0.4065       524
   macro avg     0.3141    0.3246    0.2832       524
weighted avg     0.3395    0.4065    0.3366       524



In [25]:
scores = get_metrics(y_train, y_pred_train1, y_test, y_pred_test1, "RandomForest", scores)
scores

<ipython-input-19-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.388525,0.388525,0.619293,0.318029,0.312977,0.312977,0.343680,0.232598
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.406489,0.406489,0.339546,0.336607


**AdaBoost**

In [ ]:
clf_adaboost =  AdaBoostClassifier()

In [ ]:
clf_adaboost.fit(X_train, y_train)

AdaBoostClassifier()

In [ ]:
y_pred_train2 = clf_adaboost.predict(X_train)
y_pred_test2 = clf_adaboost.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train2, y_test, y_pred_test2)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.6058    0.4044    0.4850       361
           2     0.5605    0.8373    0.6715       553
           3     0.6078    0.3039    0.4052       306

    accuracy                         0.5754      1220
   macro avg     0.5914    0.5152    0.5206      1220
weighted avg     0.5858    0.5754    0.5495      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2308    0.1529    0.1839       157
           2     0.4500    0.6456    0.5303       237
           3     0.2000    0.1231    0.1524       130

    accuracy                         0.3683       524
   macro avg     0.2936    0.3072    0.2889       524
weighted avg     0.3223    0.3683    0.3328       524



In [ ]:
scores = get_metrics(y_train, y_pred_train2, y_test, y_pred_test2, "Adaboost", scores)
scores

<ipython-input-26-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.495902,0.495902,0.716822,0.376515,0.440840,0.440840,0.364748,0.309096
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.408397,0.408397,0.342091,0.350605
2,Adaboost,0.575410,0.575410,0.585796,0.549544,0.368321,0.368321,0.322292,0.332769


**Gradient Boosting Classifier**

In [ ]:
clf_gbm = GradientBoostingClassifier()

In [ ]:
clf_gbm.fit(X=X_train, y=y_train)

GradientBoostingClassifier()

In [ ]:
y_pred_train3 = clf_gbm.predict(X_train)
y_pred_test3 = clf_gbm.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train3, y_test, y_pred_test3)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.9742    0.8366    0.9001       361
           2     0.8227    0.9982    0.9020       553
           3     0.9874    0.7712    0.8661       306

    accuracy                         0.8934      1220
   macro avg     0.9281    0.8687    0.8894      1220
weighted avg     0.9088    0.8934    0.8924      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2551    0.1592    0.1961       157
           2     0.4514    0.7046    0.5502       237
           3     0.2500    0.1077    0.1505       130

    accuracy                         0.3931       524
   macro avg     0.3188    0.3239    0.2990       524
weighted avg     0.3426    0.3931    0.3450       524



In [ ]:
scores = get_metrics(y_train, y_pred_train3, y_test, y_pred_test3, "GBM", scores)
scores

<ipython-input-26-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.495902,0.495902,0.716822,0.376515,0.440840,0.440840,0.364748,0.309096
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.408397,0.408397,0.342091,0.350605
2,Adaboost,0.575410,0.575410,0.585796,0.549544,0.368321,0.368321,0.322292,0.332769
3,GBM,0.893443,0.893443,0.908828,0.892419,0.393130,0.393130,0.342598,0.344967


**SVM Model**

In [26]:
linear_svm = SVC(kernel='linear', C=1, class_weight = "balanced")

In [27]:
linear_svm.fit(X=X_train, y= y_train)

SVC(C=1, class_weight='balanced', kernel='linear')

In [28]:
y_pred_train4 = linear_svm.predict(X_train)
y_pred_test4 = linear_svm.predict(X_test)

In [29]:
classifcation_report_train_test(y_train, y_pred_train4, y_test, y_pred_test4)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.4722    0.5180    0.4941       361
           2     0.6419    0.5316    0.5816       553
           3     0.4372    0.5229    0.4762       306

    accuracy                         0.5254      1220
   macro avg     0.5171    0.5242    0.5173      1220
weighted avg     0.5403    0.5254    0.5293      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3046    0.2930    0.2987       157
           2     0.4928    0.4346    0.4619       237
           3     0.2866    0.3615    0.3197       130

    accuracy                         0.3740       524
   macro avg     0.3613    0.3630    0.3601       524
weighted avg     0.3853    0.3740    0.3777       524



In [30]:
scores = get_metrics(y_train, y_pred_train4, y_test, y_pred_test4, "SVM", scores)
scores

<ipython-input-19-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.388525,0.388525,0.619293,0.318029,0.312977,0.312977,0.343680,0.232598
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.406489,0.406489,0.339546,0.336607
2,SVM,0.525410,0.525410,0.540349,0.529258,0.374046,0.374046,0.385273,0.377723


**KNN**

In [ ]:
knn = KNeighborsClassifier(n_neighbors=7)

In [ ]:
knn.fit(X=X_train, y=y_train)

KNeighborsClassifier(n_neighbors=7)

In [ ]:
y_pred_train5 = knn.predict(X_train)
y_pred_test5 = knn.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train5, y_test, y_pred_test5)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.5118    0.4792    0.4950       361
           2     0.5805    0.7432    0.6519       553
           3     0.5632    0.3203    0.4083       306

    accuracy                         0.5590      1220
   macro avg     0.5519    0.5142    0.5184      1220
weighted avg     0.5559    0.5590    0.5444      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3262    0.2930    0.3087       157
           2     0.4492    0.5781    0.5055       237
           3     0.2564    0.1538    0.1923       130

    accuracy                         0.3874       524
   macro avg     0.3439    0.3416    0.3355       524
weighted avg     0.3645    0.3874    0.3689       524



In [ ]:
scores = get_metrics(y_train, y_pred_train5, y_test, y_pred_test5, "KNN", scores)
scores

<ipython-input-26-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.495902,0.495902,0.716822,0.376515,0.440840,0.440840,0.364748,0.309096
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.408397,0.408397,0.342091,0.350605
2,Adaboost,0.575410,0.575410,0.585796,0.549544,0.368321,0.368321,0.322292,0.332769
3,GBM,0.893443,0.893443,0.908828,0.892419,0.393130,0.393130,0.342598,0.344967
4,SVM,0.527869,0.527869,0.524878,0.480819,0.417939,0.417939,0.362228,0.358433
5,KNN,0.559016,0.559016,0.555851,0.544363,0.387405,0.387405,0.364521,0.368858


**GNB**

In [ ]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()

In [ ]:
nb.fit(X=X_train, y=y_train)

GaussianNB()

In [ ]:
y_pred_train6 = nb.predict(X_train)
y_pred_test6 = nb.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train6, y_test, y_pred_test6)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.3481    0.3906    0.3681       361
           2     0.4755    0.6854    0.5615       553
           3     0.5000    0.0294    0.0556       306

    accuracy                         0.4336      1220
   macro avg     0.4412    0.3684    0.3284      1220
weighted avg     0.4440    0.4336    0.3774      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2848    0.2994    0.2919       157
           2     0.4438    0.6498    0.5274       237
           3     0.1667    0.0154    0.0282       130

    accuracy                         0.3874       524
   macro avg     0.2984    0.3215    0.2825       524
weighted avg     0.3274    0.3874    0.3330       524



In [ ]:
scores = get_metrics(y_train, y_pred_train6, y_test, y_pred_test6, "GNB", scores)
scores

<ipython-input-26-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.495902,0.495902,0.716822,0.376515,0.440840,0.440840,0.364748,0.309096
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.408397,0.408397,0.342091,0.350605
2,Adaboost,0.575410,0.575410,0.585796,0.549544,0.368321,0.368321,0.322292,0.332769
3,GBM,0.893443,0.893443,0.908828,0.892419,0.393130,0.393130,0.342598,0.344967
4,SVM,0.527869,0.527869,0.524878,0.480819,0.417939,0.417939,0.362228,0.358433
5,KNN,0.559016,0.559016,0.555851,0.544363,0.387405,0.387405,0.364521,0.368858
6,GNB,0.433607,0.433607,0.443977,0.377377,0.387405,0.387405,0.327423,0.332991
